in collaborative filtring, the goal is to reshape the data as a form of user-content interaction, where here the user is the user_id, the content is the book title, and the interactoin is the rating. all of that will be displayed as a table, where the rows and the columns represent the user and the book_title, and the values of the table represent the ratings.Knowing that of course not every title is rated by every user, so we're gonna have a lot of NaN values, that we'll convert em to 0s using fillna. Finally we can represent this table by sparse matrix to get rid of these 0s, and specially because sklearn algorithms handle the sparse matrices very well. 
We need to take some remarqs, for example, to better study the interactions, at least, each user has 200 ratings, and each book has 50 ratings.

In [ ]:
import numpy as np
import pandas as pd 
import chardet

In [127]:
books=pd.read_csv('BX-Books.csv',
                  sep=";",
                  error_bad_lines=False,
                  encoding='latin-1')
books.head()


C:\Users\DELL\AppData\Local\Temp\ipykernel_13160\3685496225.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books=pd.read_csv('BX-Books.csv',
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expec

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [128]:
books.shape

(271360, 8)

In [131]:
books=books[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher']]
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [132]:
books.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Publisher':'publisher'},
             inplace=True)
books

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press


In [133]:
users=pd.read_csv('BX-Users.csv',
                  sep=";",
                  error_bad_lines=False,
                  encoding='latin-1')
users.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_13160\3261301122.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users=pd.read_csv('BX-Users.csv',


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [134]:
users.rename(columns={'User-ID':'user_id','Location':'location','Age':'age'},inplace=True)
users

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [135]:
users.shape

(278858, 3)

In [136]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   278858 non-null  int64  
 1   location  278858 non-null  object 
 2   age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [137]:
ratings=pd.read_csv('BX-Book-Ratings.csv',
                    sep=";",
                    error_bad_lines=False,
                   encoding='latin-1')
ratings.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_13160\4244596143.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings=pd.read_csv('BX-Book-Ratings.csv',


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [138]:
ratings.rename(columns={'User-ID':'user_id','Book-Rating':'rating'},inplace=True)
ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [139]:
ratings.shape

(1149780, 3)

In [140]:
ratings['user_id'].value_counts().shape  #how many users, cause each users has many ratings

(105283,)

In [142]:
x=ratings['user_id'].value_counts()>200
x[x].shape  #x where x is true(let it alone default), 
           #so we have 899 users who have more than 200 ratings which help us to study them and compare each others's data
y=x[x].index


In [190]:
ratings=ratings[ratings['user_id'].isin(y)]
ratings.shape

(526356, 3)

In [156]:
ratings_with_books=ratings.merge(books,on='ISBN')
ratings_with_books

,user_id,ISBN,rating,title,author,year,publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
...,...,...,...,...,...,...,...
487666,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom
487667,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA)
487668,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim"
487669,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann


In [232]:
number_ratings=ratings_with_books.groupby('title')['rating'].count().reset_index() #grouping each title by the number of the ratings it has, we will use these as column in our final data in order to find the books which has more than 50 ratings

In [177]:
number_ratings.rename(columns={'rating':'number_of_ratings'}, inplace=True)
number_ratings

,title,number_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
160264,Ã?Â?ber die Pflicht zum Ungehorsam gegen den S...,3
160265,Ã?Â?lpiraten.,1
160266,Ã?Â?rger mit Produkt X. Roman.,1
160267,Ã?Â?stlich der Berge.,1


In [178]:
final_rating=ratings_with_books.merge(number_ratings,on='title')
final_rating

,user_id,ISBN,rating,title,author,year,publisher,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
...,...,...,...,...,...,...,...,...
487666,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom,1
487667,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),1
487668,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",1
487669,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,1


In [194]:
final_rating=final_rating[final_rating['number_of_ratings']>=50]
final_rating.reset_index(inplace=True)

ValueError: cannot insert level_0, already exists

In [195]:
final_rating

,level_0,index,user_id,ISBN,rating,title,author,year,publisher,number_of_ratings
0,0,0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,1,1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,2,2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,3,3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,4,4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
...,...,...,...,...,...,...,...,...,...,...
61848,61848,236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,50
61849,61849,236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
61850,61850,236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
61851,61851,236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50


In [200]:
final_rating.drop_duplicates(['title','user_id'],inplace=True)  #qui ont le meme title et l'id en mm temps 

In [202]:
book_pivot=final_rating.pivot_table(columns='user_id',index='title',values='rating')  #a lot of NaN (because not every title is rated by every user_id)

In [216]:
book_pivot.fillna(0,inplace=True)   #lot of zeros ---> sparse matrix

In [241]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [215]:
final_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59850 entries, 0 to 61852
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   level_0            59850 non-null  int64 
 1   index              59850 non-null  int64 
 2   user_id            59850 non-null  int64 
 3   ISBN               59850 non-null  object
 4   rating             59850 non-null  int64 
 5   title              59850 non-null  object
 6   author             59850 non-null  object
 7   year               59850 non-null  object
 8   publisher          59850 non-null  object
 9   number_of_ratings  59850 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 5.0+ MB


In [227]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(book_pivot)
print(book_sparse)

  (0, 0)	9.0
  (0, 16)	8.0
  (0, 37)	9.0
  (0, 42)	8.0
  (0, 59)	7.0
  (0, 135)	9.0
  (0, 156)	10.0
  (0, 170)	9.0
  (0, 187)	8.0
  (0, 220)	10.0
  (0, 221)	10.0
  (0, 245)	10.0
  (0, 277)	10.0
  (0, 359)	10.0
  (0, 370)	9.0
  (0, 389)	10.0
  (0, 477)	9.0
  (0, 491)	7.0
  (0, 507)	9.0
  (0, 554)	10.0
  (0, 604)	9.0
  (0, 611)	8.0
  (0, 695)	9.0
  (0, 734)	9.0
  (0, 762)	8.0
  :	:
  (740, 484)	8.0
  (740, 681)	9.0
  (741, 16)	8.0
  (741, 26)	7.0
  (741, 86)	10.0
  (741, 111)	5.0
  (741, 150)	6.0
  (741, 233)	7.0
  (741, 290)	8.0
  (741, 303)	9.0
  (741, 322)	10.0
  (741, 367)	10.0
  (741, 384)	9.0
  (741, 446)	7.0
  (741, 450)	9.0
  (741, 485)	7.0
  (741, 498)	10.0
  (741, 505)	8.0
  (741, 544)	10.0
  (741, 592)	8.0
  (741, 700)	10.0
  (741, 713)	8.0
  (741, 745)	10.0
  (741, 830)	9.0
  (741, 880)	8.0


In [223]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [236]:
from sklearn.neighbors import NearestNeighbors
mdl=NearestNeighbors(algorithm='brute')

In [245]:
mdl.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
book_pivot.iloc[237,:].values.reshape(1,-1)  # this is not the code it's the parameter of the next synyaxe, it's just to explain it

The code "book_pivot.iloc[237,:].values.reshape(1,-1)" is used to select a specific row from the user-item matrix "book_pivot" and reshape it into a 2-dimensional numpy array with one row and an unspecified number of columns.

Let's break down the code:

"book_pivot" is a pandas DataFrame that represents the user-item matrix for a book recommendation system. Each row corresponds to a book, each column corresponds to a user, and each entry represents the user's rating of the book.

".iloc[237,:]" selects the 238th row (indexed from 0) of the DataFrame "book_pivot". This corresponds to the ratings of all users for the 238th book.

".values" returns the values of the selected row as a numpy array.

".reshape(1,-1)" reshapes the 1-dimensional numpy array into a 2-dimensional numpy array with one row and an unspecified number of columns (-1). This is done to ensure that the array has the correct shape to be passed as input to the "mdl.kneighbors" method.

Overall, the code "book_pivot.iloc[237,:].values.reshape(1,-1)" selects the ratings of all users for a specific book in the user-item matrix "book_pivot" and reshapes them into a 2-dimensional numpy array. This 2-dimensional array can then be used as input to the k-NN algorithm to find the k-nearest neighbors of the selected book.

In [249]:
distances, suggestions=mdl.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)

The code "distances, suggestions=mdl.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)" is also used to find the k-nearest neighbors of a book in the user-item matrix "book_pivot", using a k-Nearest Neighbors (k-NN) algorithm implemented in the scikit-learn library.

Let's break down the code:

"mdl" is an instance of the KNeighborsClassifier class in scikit-learn, which has been fit to the user-item matrix "book_pivot".

"mdl.kneighbors" is a method of the KNeighborsClassifier class that returns the k-nearest neighbors of a given data point, along with their distances. In this case, the input data point is the row of the user-item matrix corresponding to a book, which has been reshaped into a 2-dimensional numpy array using the ".values.reshape(1,-1)" syntax. The "n_neighbors" argument is set to 6, indicating that we want to find the 6 nearest neighbors of the input data point.

"distances" and "suggestions" are the outputs of the "mdl.kneighbors" method. "distances" is a numpy array containing the distances between the input data point and its 6 nearest neighbors, and "suggestions" is a numpy array containing the indices of the 6 nearest neighbors in the user-item matrix.

Overall, the code "distances, suggestions=mdl.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)" is used to find the 6 nearest neighbors of a book in the user-item matrix "book_pivot".-------->>>> This information can be used to make recommendations for similar books to users who have rated the selected book highly  <<<<------------.

In [250]:
suggestions

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [261]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [264]:
print(book_pivot.index[237])

Harry Potter and the Chamber of Secrets (Book 2)


in the case of reshape(1,-1), the kneighbors require a 2 demensional array, and that what do reshape, it doestn mean 2 columns or rows, it means [[1,50]] for example, its just about the shape. and the -1 is to let reshape findout automatically from the right format.

In [269]:
np.where(book_pivot.index=='Animal Farm')   #this how we find the row of any book we want to predict similar books to it.(input of the predictor) 

(array([54], dtype=int64),)

In [270]:
np.where(book_pivot.index=='Animal Farm')[0] #reduce one dimnsion in the view

array([54], dtype=int64)

In [272]:
np.where(book_pivot.index=='Animal Farm')[0][0]  #reduce 2 dimension

54

In [301]:
def book_recommander(book_name):
    ind=np.where(book_pivot.index==book_name)[0][0]
    distances, suggestions=mdl.kneighbors(book_pivot.iloc[ind,:].values.reshape(1,-1),n_neighbors=6)
    
    for i in range(len(suggestions)):
        if i==0:
            print(f'the suggestion for the book <{book_name}> are :')
        if not i:
            print(book_pivot.index[suggestions[i]])

In [302]:
book_recommander('And Then You Die')

the suggestion for the book <And Then You Die> are :
Index(['And Then You Die', 'Long After Midnight', 'No Safe Place', 'Exclusive',
       'The Most Wanted', 'Executive Orders (Jack Ryan Novels)'],
      dtype='object', name='title')
